In [4]:
library('transformeR')
library(raster)


    _______   ____  ___________________  __  ________ 
   / ___/ /  / /  |/  / __  /_  __/ __/ / / / / __  / 
  / /  / /  / / /|_/ / /_/ / / / / __/ / /_/ / /_/_/  
 / /__/ /__/ / /  / / __  / / / / /__ /___  / / \ \ 
 \___/____/_/_/  /_/_/ /_/ /_/  \___/    /_/\/   \_\ 
 
      github.com/SantanderMetGroup/climate4R



transformeR version 2.1.3 (2021-08-04) is loaded


Get the latest stable version (2.1.5) using <devtools::install_github('SantanderMetGroup/transformeR')>

Please see 'citation("transformeR")' to cite this package.

Loading required package: sp



In [5]:
#archivosdf <- list.files(path = '../3.Datos generados/dataframes', pattern = "^df_grid_", full.names = TRUE)
#archivosdf
#archivos_grid <- list.files(path = '../1.Upscaling/upscaling_ba', full.names = TRUE)
archivos_raster <- list.files(path = '../3.Datos generados/raster', full.names = TRUE)
#archivos_raster

In [6]:
df_grid_25_sinInterpolar <- get(load('../3.Datos generados/dataframes/df_grid_25.Rdata'))

Con vecinos más cercanos

In [7]:
lista_variables <- c('grid_05','grid_1','grid_15','grid_2','grid_25','grid_3', 'grid_4','grid_5')
lista_df_interpoladas <- list()
for (i in 1:length(archivos_raster)){
    lista_interpoladas <- list()
    for (j in seq(1,9)){
        grid_raster <- raster(archivos_raster[i], band = j)  
        #creamos un raster vacío a 2.5
        grid_raster_2.5 <- raster(extent(grid_raster), res = 2.5)
        #realizamos interpolación por vecinos cercanos (ngb)
        raster_interp <- resample(grid_raster, grid_raster_2.5, method = "ngb")
        vector_raster_interp <- as.vector(raster_interp)
        columna = paste0('layer_',j)
        lista_interpoladas[[columna]] <- vector_raster_interp 
    }
    coordenadas_interpolada <- coordinates(raster_interp)
    df_interpoladas <- data.frame(lista_interpoladas)
    df_interpoladas <- cbind(coordenadas_interpolada, df_interpoladas)
    colnames(df_interpoladas) <- c('X','Y','FsOrNot','MainStart','MainEnd','SencondStart','SencondEnd','Length','C','P','FBA')
    var_name = paste0('df_interpoladas_',lista_variables[i])
    lista_df_interpoladas[[var_name]] <- df_interpoladas
}

In [8]:
vector_grids <- c('grid_05','grid_1','grid_15','grid_2','grid_25','grid_3', 'grid_4','grid_5')
variablesComparar <- c('FsOrNot','MainStart','MainEnd','SencondStart','SencondEnd','Length','C','P','FBA')
lista_grids_MAES <- list()
for (i in 1:length(lista_df_interpoladas)){
    grid <- vector_grids[i]
    interpolada <- lista_df_interpoladas[[i]]
    MAE_s <- c()
    maxError <- c()
    for (var in variablesComparar){
        var_inter <- unlist(interpolada[var])
        names(var_inter) <- NULL
        var_grid_25_sinInterpolar <-unlist(lista_df_interpoladas$'df_interpoladas_grid_25'[var])
        names(var_grid_25_sinInterpolar) <- NULL
        diff_var <- c()
        for (item in 1:length(var_inter)){
            if (is.na(var_inter[item]) & is.na(var_grid_25_sinInterpolar[item])){
                diff_var <- c(diff_var ,0)
            }else{
                diff_var <- c(diff_var, abs(var_inter[item] - var_grid_25_sinInterpolar[item]) )
            }
        }
    diff_var <- na.omit(diff_var) 
    MAE_s <- c(MAE_s, mean(diff_var))
    maxError <- c(maxError, max(diff_var))
    }
    MAE_s_normalizado <- MAE_s/maxError
    names(MAE_s_normalizado) <- variablesComparar
    if (vector_grids[i] == 'grid_25'){
        lista_grids_MAES[[vector_grids[i]]] <- MAE_s
    }else{
        lista_grids_MAES[[vector_grids[i]]] <- MAE_s_normalizado
    }
}

In [9]:
t(data.frame(lista_grids_MAES))

,FsOrNot,MainStart,MainEnd,SencondStart,SencondEnd,Length,C,P,FBA
grid_05,0.08834877,0.02825821,0.04069830,0.012523946,0.008596743,0.05775862,0.12493220,0.06166349,0.27986740
grid_1,0.05304784,0.01911147,0.03625993,0.005383580,0.003545916,0.04058306,0.06243892,0.04736081,0.04049946
grid_15,0.03288966,0.01345368,0.02945048,0.005130028,0.004135949,0.02976408,0.04377659,0.03603289,0.04236243
grid_2,0.02662037,0.01071956,0.02362808,0.004345190,0.003410469,0.02546572,0.03717047,0.03330795,0.03650922
grid_25,0.00000000,0.00000000,0.00000000,0.000000000,0.000000000,0.00000000,0.00000000,0.00000000,0.00000000
grid_3,0.03182870,0.01283305,0.02850984,0.005653458,0.003985433,0.03183458,0.04608705,0.04157228,0.04257019
grid_4,0.04812886,0.01627686,0.03613703,0.006307115,0.004907292,0.04126276,0.06190737,0.05174731,0.04595209
grid_5,0.06172840,0.01720432,0.03901577,0.006273058,0.004410926,0.03910362,0.07110289,0.05548743,0.06296611


In [10]:
library(xtable)
table <- xtable(t(data.frame(lista_grids_MAES)))
print(table, include.rownames = FALSE)

% latex table generated in R 3.6.3 by xtable 1.8-4 package
% Wed Sep 20 08:07:55 2023
\begin{table}[ht]
\centering
\begin{tabular}{rrrrrrrrr}
  \hline
FsOrNot & MainStart & MainEnd & SencondStart & SencondEnd & Length & C & P & FBA \\ 
  \hline
0.09 & 0.03 & 0.04 & 0.01 & 0.01 & 0.06 & 0.12 & 0.06 & 0.28 \\ 
  0.05 & 0.02 & 0.04 & 0.01 & 0.00 & 0.04 & 0.06 & 0.05 & 0.04 \\ 
  0.03 & 0.01 & 0.03 & 0.01 & 0.00 & 0.03 & 0.04 & 0.04 & 0.04 \\ 
  0.03 & 0.01 & 0.02 & 0.00 & 0.00 & 0.03 & 0.04 & 0.03 & 0.04 \\ 
  0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 \\ 
  0.03 & 0.01 & 0.03 & 0.01 & 0.00 & 0.03 & 0.05 & 0.04 & 0.04 \\ 
  0.05 & 0.02 & 0.04 & 0.01 & 0.00 & 0.04 & 0.06 & 0.05 & 0.05 \\ 
  0.06 & 0.02 & 0.04 & 0.01 & 0.00 & 0.04 & 0.07 & 0.06 & 0.06 \\ 
   \hline
\end{tabular}
\end{table}


Con bilinear

In [344]:
lista_variables <- c('grid_05','grid_1','grid_15','grid_2','grid_25','grid_3', 'grid_4','grid_5')
lista_df_interpoladas <- list()
for (i in 1:length(archivos_raster)){
    lista_interpoladas <- list()
    for (j in seq(1,9)){
        grid_raster <- raster(archivos_raster[i], band = j)  
        #creamos un raster vacío a 2.5
        grid_raster_2.5 <- raster(extent(grid_raster), res = 2.5)
        #realizamos interpolación por vecinos cercanos (ngb)
        raster_interp <- resample(grid_raster, grid_raster_2.5, method = "bilinear")
        vector_raster_interp <- as.vector(raster_interp)
        columna = paste0('layer_',j)
        lista_interpoladas[[columna]] <- vector_raster_interp 
    }
    coordenadas_interpolada <- coordinates(raster_interp)
    df_interpoladas <- data.frame(lista_interpoladas)
    df_interpoladas <- cbind(coordenadas_interpolada, df_interpoladas)
    colnames(df_interpoladas) <- c('X','Y','FsOrNot','MainStart','MainEnd','SencondStart','SencondEnd','Length','C','P','FBA')
    var_name = paste0('df_interpoladas_',lista_variables[i])
    lista_df_interpoladas[[var_name]] <- df_interpoladas
}

In [346]:
vector_grids <- c('grid_05','grid_1','grid_15','grid_2','grid_25','grid_3', 'grid_4','grid_5')
variablesComparar <- c('FsOrNot','MainStart','MainEnd','SencondStart','SencondEnd','Length','C','P','FBA')
lista_grids_MAES <- list()
for (i in 1:length(lista_df_interpoladas)){
    grid <- vector_grids[i]
    interpolada <- lista_df_interpoladas[[i]]
    MAE_s <- c()
    for (var in variablesComparar){
        var_inter <- unlist(interpolada[var])
        names(var_inter) <- NULL
        var_grid_25_sinInterpolar <-unlist(lista_df_interpoladas$'df_interpoladas_grid_25'[var])
        names(var_grid_25_sinInterpolar) <- NULL
        diff_var <- c()
        for (item in 1:length(var_inter)){
            if (is.na(var_inter[item]) & is.na(var_grid_25_sinInterpolar[item])){
                diff_var <- c(diff_var ,0)
            }else{
                diff_var <- c(diff_var, abs(var_inter[item] - var_grid_25_sinInterpolar[item]) )
            }
        }
    diff_var <- na.omit(diff_var)    
    MAE_var = mean(diff_var)
    MAE_s <- c(MAE_s, MAE_var)
    }
    names(MAE_s) <- variablesComparar
    lista_grids_MAES[[vector_grids[i]]] <- MAE_s
}

In [347]:
t(data.frame(lista_grids_MAES))

,FsOrNot,MainStart,MainEnd,SencondStart,SencondEnd,Length,C,P,FBA
grid_05,0.09111032,0.4266898,0.4609441,0.17109992,0.12693339,0.5702300,0.07349209,0.1615045,0.299206427
grid_1,0.05365668,0.2339334,0.3705870,0.08236500,0.05539743,0.3801183,0.05327888,0.1317174,0.011157023
grid_15,0.03680127,0.1702487,0.3340071,0.07689754,0.05477719,0.3036438,0.04154976,0.1119714,0.007999769
grid_2,0.03251742,0.1640000,0.3228464,0.08242287,0.06060365,0.2778054,0.03722217,0.1082916,0.010575980
grid_25,0.00000000,0.0000000,0.0000000,0.00000000,0.00000000,0.0000000,0.00000000,0.0000000,0.000000000
grid_3,0.04201323,0.2012983,0.3876081,0.09580311,0.07239528,0.3347066,0.05007797,0.1315018,0.022683365
grid_4,0.05798298,0.2312029,0.4679415,0.12115645,0.08787251,0.4188026,0.06603080,0.1623885,0.033004167
grid_5,0.07767077,0.2534552,0.4915593,0.12443753,0.08922049,0.4585908,0.08245383,0.1782202,0.044799817


### Visualizaciones

In [32]:
lista_variables <- c('grid_05','grid_2','grid_3','grid_5')
variablesComparar <- c('Length','C','P')
lista_df_interpoladas_viz <- list(lista_df_interpoladas$'df_interpoladas_grid_05', lista_df_interpoladas$'df_interpoladas_grid_2', lista_df_interpoladas$'df_interpoladas_grid_3', lista_df_interpoladas$'df_interpoladas_grid_5')
lista_grids <- list()
for (i in 1:length(lista_df_interpoladas_viz)){
    grid <- vector_grids[i]
    interpolada <- lista_df_interpoladas_viz[[i]]
    diff_list <- list()
    for (var in variablesComparar){
        var_inter <- unlist(interpolada[var])
        names(var_inter) <- NULL
        var_grid_25_sinInterpolar <-unlist(lista_df_interpoladas$'df_interpoladas_grid_25'[var])
        names(var_grid_25_sinInterpolar) <- NULL
        diff_var <- c()
        for (item in 1:length(var_inter)){
            if (is.na(var_inter[item]) & is.na(var_grid_25_sinInterpolar[item])){
                diff_var <- c(diff_var ,0)
            }else{
                diff_var <- c(diff_var, abs(var_inter[item] - var_grid_25_sinInterpolar[item]) )
            }
        }
    diff_var <- na.omit(diff_var)    
    diff_list[[var]] <- diff_var
    }
    lista_grids[[vector_grids[i]]] <- diff_list
}

In [34]:
library("visualizeR")

quantity2clim <- function(quantity, what, ref.grid, backperm = NULL) {
  if(!is.null(backperm)){quantity <- quantity[backperm]}
  mat <- matrix(quantity, nrow = 1)  
  ref.grid$Data <- mat2Dto3Darray(mat, x = ref.grid$xyCoords$x , y = ref.grid$xyCoords$y)
  attr(ref.grid$Data, "climatology:fun") <- what
  return(ref.grid)
}


In [42]:
grid_25 <- get(load('../1.Upscaling/upscaling_ba/ba_grid_25.Rdata'))

In [46]:
MAE_C_clim05 <- quantity2clim(quantity = lista_grids$grid_05$C, what = 'MAE_s05C', ref.grid = grid_25, backperm = NULL)
MAE_P_clim05 <- quantity2clim(quantity = lista_grids$grid_05$P, what = 'MAE_s05P', ref.grid = grid_25, backperm = NULL)
MAE_L_clim05 <- quantity2clim(quantity = lista_grids$grid_05$Length, what = 'MAE_s05L', ref.grid = grid_25, backperm = NULL)

MAE_C_clim5 <- quantity2clim(quantity = lista_grids$grid_05$C, what = 'MAE_s5C', ref.grid = grid_25, backperm = NULL)
MAE_P_clim5 <- quantity2clim(quantity = lista_grids$grid_05$P, what = 'MAE_s5P', ref.grid = grid_25, backperm = NULL)
MAE_L_clim5 <- quantity2clim(quantity = lista_grids$grid_05$Length, what = 'MAE_s5L', ref.grid = grid_25, backperm = )

Warning message in matrix(mat2D[i, ], ncol = length(x), nrow = length(y)):
“data length [10092] is not a sub-multiple or multiple of the number of rows [144]”
Warning message in matrix(mat2D[i, ], ncol = length(x), nrow = length(y)):
“data length [10092] is not a sub-multiple or multiple of the number of rows [144]”
